In [140]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [141]:

def prepare_chart_data(df, column1, column2, diff_column_name, avg_column_name, years=10):
    """
    Calculate the difference between two columns, create a new column for the difference,
    and add another column with the average of the difference over the past n years.

    Parameters:
    df (DataFrame): The input DataFrame.
    column1 (str): The name of the first column.
    column2 (str): The name of the second column.
    diff_column_name (str): The name of the new column to store the difference.
    avg_column_name (str): The name of the new column to store the average.
    years (int): Number of years to calculate the average (default is 10).

    Returns:
    DataFrame: The updated DataFrame with the new columns.
    """
    # Calculate the difference and create a new column
    df[diff_column_name] = df[column1] - df[column2]
    
    # Ensure the "Date" column is in datetime format
    df["Date"] = pd.to_datetime(df["Date"])
    
    # Filter for the past `years` years
    filtered_df = df[df["Date"] >= (df["Date"].max() - pd.DateOffset(years=years))]
    
    # Calculate the average for the past `years` years
    avg_value = filtered_df[diff_column_name].mean()
    
    # Add a new column with the constant average value for the filtered data
    filtered_df[avg_column_name] = avg_value
    
    return filtered_df[["Date", diff_column_name, avg_column_name]]

def plot_interest_rates(df, date_col, real_rate_col, avg_rate_col, output_path):
    """
    Creates a beautiful line chart showing Real Interest Rate and 10-Year Average,
    and saves it as a PNG file.

    Parameters:
    df (DataFrame): The input DataFrame.
    date_col (str): The name of the date column.
    real_rate_col (str): The name of the Real Interest Rate column.
    avg_rate_col (str): The name of the 10-Year Average column.
    output_path (str): The path where the PNG file will be saved.

    Returns:
    None
    """
    # Paths to your Montserrat font files
    font_path = './assets/Montserrat/static/Montserrat-Regular.ttf'
    font_path_bold = './assets/Montserrat/static/Montserrat-Black.ttf'

    # Register the font properties
    prop = fm.FontProperties(fname=font_path)
    prop_bold = fm.FontProperties(fname=font_path_bold)

    # Convert the date column to datetime for better plotting
    df[date_col] = pd.to_datetime(df[date_col])

    # Create the plot
    plt.figure(figsize=(4.14, 2.5))  # Adjusted dimensions for consistency

    real_interest_color = "#003366"
    ten_yr_avg_color = "#FFA500"

    # Plot Real Interest Rate
    plt.plot(df[date_col], df[real_rate_col], label=f'Rate', linewidth=2, 
            alpha=0.8, color=real_interest_color)

    # Add the last value to the Real Interest Rate line
    last_x = df[date_col].iloc[0]
    last_y = df[real_rate_col].iloc[0]
    plt.text(last_x, last_y, f'{last_y:.2f}', fontsize=8, color="white", 
             ha='center', va='center', fontproperties=prop_bold, 
             bbox=dict(facecolor=real_interest_color, edgecolor='none', boxstyle='round,pad=0.3'))


    # Plot 10-Year Average
    plt.plot(df[date_col], df[avg_rate_col], label='10-Year Average', linewidth=2, 
             linestyle='--', alpha=0.8, color=ten_yr_avg_color)

    # Add title and labels
    plt.title(f'{real_rate_col}', fontsize=10, fontproperties=prop_bold)
    plt.xlabel('Date', fontsize=8, fontproperties=prop)
    plt.ylabel('Rate (%)', fontsize=8, fontproperties=prop)

    # Add grid, legend, and adjust ticks
    plt.grid(visible=True, linestyle='--', alpha=0.6)
    plt.legend(fontsize=8, prop=prop, loc='lower left')
    plt.xticks(fontsize=8, rotation=45, fontproperties=prop)
    plt.yticks(fontsize=8, fontproperties=prop)

    # Tight layout for better spacing
    plt.tight_layout()

    # Save the plot to the specified path
    plt.savefig(output_path, format='png', dpi=300, bbox_inches='tight')

    # Close the plot to free memory
    plt.close()


In [142]:
# Table Data

# Data from the image
data = {
    "Category": [
        "Real Central Bank Rate",
        "Central Bank differential vs Fed",
        "Inflation differential vs US",
        "Real Central Rate vs Real Fed Rate",
        "2yr Rate differential vs US",
        "10yr Rate differential vs US"
    ],
    "Now": [7.4, 7.7, 2.2, 5.5, 11.3, 10.4],
    "Average": [3.8, 8.5, 3.4, 5.1, 8.7, 9.0],
    "Delta": [3.6, -0.8, -1.2, 0.4, 2.5, 1.4]
}

# Creating the DataFrame
df = pd.DataFrame(data)

# Display the table
print(df)

export_path = "../data/processed/"
file_name = "monetary_policy.csv"

df.to_csv(os.path.join(export_path, file_name), index=False)

                             Category   Now  Average  Delta
0              Real Central Bank Rate   7.4      3.8    3.6
1    Central Bank differential vs Fed   7.7      8.5   -0.8
2        Inflation differential vs US   2.2      3.4   -1.2
3  Real Central Rate vs Real Fed Rate   5.5      5.1    0.4
4         2yr Rate differential vs US  11.3      8.7    2.5
5        10yr Rate differential vs US  10.4      9.0    1.4


In [143]:
# Creating Charts
dataset = pd.read_excel("../data/raw/data.xlsx", sheet_name=0).iloc[:,0:8].dropna()

# Preparing Dataset
dataset["US_Real_Rate"] =  dataset["US_Funds"] - dataset["US_Inflation"]
dataset["BR_Real_Rate"] =  dataset["BR_Selic"] - dataset["BR_Inflation"]

dataset

,Year,Month,Date,BR_Selic,BR_Inflation,US_Funds,US_Inflation,BRL,US_Real_Rate,BR_Real_Rate
0,2024.0,12.0,2024-12-31,12.25,4.76,4.625,2.6,6.2896,2.025,7.49
1,2024.0,11.0,2024-11-30,11.25,4.76,4.625,2.6,5.9730,2.025,6.49
2,2024.0,10.0,2024-10-31,10.75,4.76,4.875,2.6,5.7867,2.275,5.99
3,2024.0,9.0,2024-09-30,10.75,4.42,4.875,2.4,5.4482,2.475,6.33
4,2024.0,8.0,2024-08-31,10.50,4.24,5.375,2.5,5.6103,2.875,6.26
...,...,...,...,...,...,...,...,...,...,...
234,2005.0,6.0,2005-06-30,19.75,7.27,3.250,2.5,2.3349,0.750,12.48
235,2005.0,5.0,2005-05-31,19.75,8.05,3.000,2.8,2.4085,0.200,11.70
236,2005.0,4.0,2005-04-30,19.50,8.07,2.750,3.5,2.5272,-0.750,11.43
237,2005.0,3.0,2005-03-31,19.25,7.54,2.750,3.1,2.6790,-0.350,11.71


In [144]:
export_path = "../report/images/"

# Prepare Chart: Brazil Real Interest Rate
export_name_real_rate = "real_interest_rate_br.png"
chart_name_real_rate = "Brazil Real Interest Rate"

real_rate_chart_data = prepare_chart_data(dataset, 'BR_Selic', 'BR_Inflation', chart_name_real_rate, '10-Year Average')
plot_interest_rates(real_rate_chart_data, 'Date', chart_name_real_rate, '10-Year Average', export_path + export_name_real_rate)

# Prepare Chart: Brazil Differential Nominal Rate (Vs Fed)
export_name_nominal_diff = "nominal_fed.png"
chart_name_nominal_diff = "Brazil Differential Nominal Rate (Vs Fed)"

nominal_diff_chart_data = prepare_chart_data(dataset, 'BR_Selic', 'US_Funds', chart_name_nominal_diff, '10-Year Average')
plot_interest_rates(nominal_diff_chart_data, 'Date', chart_name_nominal_diff, '10-Year Average', export_path + export_name_nominal_diff)

# Prepare Chart: Brazil Differential Real Rate (Vs. Fed)
export_name_real_diff = "real_diff.png"
chart_name_real_diff = "Brazil Differential Real Rate (Vs. Fed)"

real_diff_chart_data = prepare_chart_data(dataset, 'BR_Real_Rate', 'US_Real_Rate', chart_name_real_diff, '10-Year Average')
plot_interest_rates(real_diff_chart_data, 'Date', chart_name_real_diff, '10-Year Average', export_path + export_name_real_diff)

# Prepare Chart: Brazil Inflation Differential (Vs. US)
export_name_inflation_diff = "diff_inflation.png"
chart_name_inflation_diff = "Brazil Inflation Differential (Vs. US)"

inflation_diff_chart_data = prepare_chart_data(dataset, 'BR_Inflation', 'US_Inflation', chart_name_inflation_diff, '10-Year Average')
plot_interest_rates(inflation_diff_chart_data, 'Date', chart_name_inflation_diff, '10-Year Average', export_path + export_name_inflation_diff)

/var/folders/ym/3qhchx8d19q8jp5fvb95q9vw0000gn/T/ipykernel_5313/4273844340.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[avg_column_name] = avg_value
/var/folders/ym/3qhchx8d19q8jp5fvb95q9vw0000gn/T/ipykernel_5313/4273844340.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[avg_column_name] = avg_value
/var/folders/ym/3qhchx8d19q8jp5fvb95q9vw0000gn/T/ipykernel_5313/4273844340.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [145]:
def collect_first_row_from_charts(categories, *datasets):
    """
    Collects the first row from each dataset and creates a table.

    Args:
        categories (list): A list of category names for the datasets.
        datasets (list): Datasets to collect the first row from.

    Returns:
        pd.DataFrame: A DataFrame containing the first row from each dataset.
    """
    table_data = {
        "Category": categories,
        "Now": [],
        "Average": []
    }

    for data in datasets:
        if not data.empty:
            first_row = data.iloc[0]
            table_data["Now"].append(first_row["Now"] if "Now" in first_row else None)
            table_data["Average"].append(first_row["Average"] if "Average" in first_row else None)
        else:
            table_data["Now"].append(None)
            table_data["Average"].append(None)

    return pd.DataFrame(table_data)

# Categories List
categories = [
    "Brazil Real Interest Rate",
    "Brazil Differential Nominal Rate (Vs Fed)",
    "Brazil Differential Real Rate (Vs. Fed)",
    "Brazil Inflation Differential (Vs. US)"
]

# Create the Table
result_table = collect_first_row_from_charts(categories, real_rate_chart_data, nominal_diff_chart_data, real_diff_chart_data, inflation_diff_chart_data)
print(result_table)

                                    Category   Now Average
0                  Brazil Real Interest Rate  None    None
1  Brazil Differential Nominal Rate (Vs Fed)  None    None
2    Brazil Differential Real Rate (Vs. Fed)  None    None
3     Brazil Inflation Differential (Vs. US)  None    None
